In [1]:
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup as bs
from pprint import pprint
import numpy as np

In [2]:
from webdriver_manager.chrome import ChromeDriverManager

In [5]:
wd = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\redoc\AppData\Local\Temp\ipykernel_157988\3832095755.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(ChromeDriverManager().install())


# 1. Action Films

In [6]:
url = "https://en.wikipedia.org/wiki/Lists_of_action_films"

In [7]:
wd.get(url)

In [8]:
wd.find_element("xpath", "//*[@id='mw-content-text']/div[1]/ul[1]/li[1]/a").get_attribute("href")

'https://en.wikipedia.org/wiki/List_of_action_films_of_the_1960s'

In [9]:
wd.find_element("xpath", "//*[@id='mw-content-text']/div[1]/ul[1]/li[2]/a").get_attribute("href")

'https://en.wikipedia.org/wiki/List_of_action_films_of_the_1970s'

## Generate Decade List

In [10]:
np.arange(1960, 2021, 10)

array([1960, 1970, 1980, 1990, 2000, 2010, 2020])

In [9]:
url_action_list = []

for i in np.arange(1960, 2021, 10):
    url_ = 'https://en.wikipedia.org/wiki/List_of_action_films_of_the_'+str(i)+'s'
    url_action_list.append(url_)

url_action_list

['https://en.wikipedia.org/wiki/List_of_action_films_of_the_1960s',
 'https://en.wikipedia.org/wiki/List_of_action_films_of_the_1970s',
 'https://en.wikipedia.org/wiki/List_of_action_films_of_the_1980s',
 'https://en.wikipedia.org/wiki/List_of_action_films_of_the_1990s',
 'https://en.wikipedia.org/wiki/List_of_action_films_of_the_2000s',
 'https://en.wikipedia.org/wiki/List_of_action_films_of_the_2010s',
 'https://en.wikipedia.org/wiki/List_of_action_films_of_the_2020s']

## Check Each Website for Data XPaths

### Base XPath

//*[@id="mw-content-text"]/div[1]/table[2]/tbody/tr[index]

Ball park I am going to use 300 movies per decade as the gauge.

In [10]:
row_xpath = np.arange(0,300,1)

In [11]:
row_xpath

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

### Title XPaths

//*[@id="mw-content-text"]/div[1]/table[2]/tbody/tr[3]/td[1]/i/b/a

//*[@id="mw-content-text"]/div[1]/table[2]/tbody/tr[4]/td[1]/i/b/a

//*[@id="mw-content-text"]/div[1]/table[2]/tbody/tr[8]/td[1]/i/b/a

//*[@id="mw-content-text"]/div[1]/table[2]/tbody/tr[254]/td[1]/i/b/a

tr[ ] indicates the indexing of rows, BUT not all rows have titles - so use TRY-EXCEPT

All Xpath for Title ends with **td[1]/i/b/a**


In [12]:
wd.get(url_action_list[1])

In [13]:
title_ = wd.find_element("xpath", "//*[@id='mw-content-text']/div[1]/table[2]/tbody/tr[254]/td[1]/i/b/a").text

In [14]:
title_

'Ten Tigers from Kwangtung'

### Country

//*[@id="mw-content-text"]/div[1]/table[2]/tbody/tr[3]/td[4]

//*[@id="mw-content-text"]/div[1]/table[2]/tbody/tr[18]/td[4]

//*[@id="mw-content-text"]/div[1]/table[2]/tbody/tr[34]/td[4]

Xpath for country ends with **td[4]**

In [15]:
title_ = wd.find_element("xpath", "//*[@id='mw-content-text']/div[1]/table[2]/tbody/tr[254]/td[4]").text

title_

'Hong Kong'

### Subgenre

//*[@id="mw-content-text"]/div[1]/table[2]/tbody/tr[5]/td[5]

//*[@id="mw-content-text"]/div[1]/table[2]/tbody/tr[17]/td[5]

Xpath for Subgenre ends with **td[5]**

In [16]:
title_ = wd.find_element("xpath", "//*[@id='mw-content-text']/div[1]/table[2]/tbody/tr[17]/td[5]").text

title_

'Martial arts film[7]'

---

In [17]:
action_movies = []

In [18]:
for i in url_action_list:
    wd.get(i)

    for row_ in row_xpath:
        movie = {}
        try:
            movie['Title'] = wd.find_element("xpath", "//*[@id='mw-content-text']/div[1]/table[2]/tbody/tr["+str(row_)+"]/td[1]/i/b/a").text
            movie['Country'] = wd.find_element("xpath", "//*[@id='mw-content-text']/div[1]/table[2]/tbody/tr["+str(row_)+"]/td[4]").text
            movie['Subgenre'] = wd.find_element("xpath", "//*[@id='mw-content-text']/div[1]/table[2]/tbody/tr["+str(row_)+"]/td[5]").text
        except:
            pass
        action_movies.append(movie)

In [ ]:
action_movies_df = pd.DataFrame(action_movies)

In [ ]:
action_movies_df.dropna(axis = 0, how = 'all', inplace = True)

In [ ]:
action_movies_df.to_csv("Action_Movie_Scrape.csv")

---

In [11]:
df = pd.read_csv('Action_Movie_Scrape.csv')

In [12]:
df['Genre']='Action'

In [13]:
df.head()

,Unnamed: 0,Title,Country,Subgenre,Genre
0,3,Dr. No,United Kingdom\nUnited States,[3],Action
1,4,Shinobi no Mono,Japan,Martial arts film[4],Action
2,6,From Russia with Love,United Kingdom\nUnited States,[5],Action
3,8,Goldfinger,United Kingdom\nUnited States,[6],Action
4,9,Greed in the Sun,France\nItaly,Action comedy[7],Action


In [14]:
df.to_csv('Action_Movie_Scrape_Final.csv')

---

## Remaining Categories

https://en.wikipedia.org/wiki/Lists_of_adventure_films

https://en.wikipedia.org/wiki/Lists_of_comedy_films

https://en.wikipedia.org/wiki/List_of_drama_films

https://en.wikipedia.org/wiki/Lists_of_horror_films

Wiki basically has all the film data tracing back till the 1920s. Compare all their URLs

https://en.wikipedia.org/wiki/List_of_adventure_films_of_the_1920s

https://en.wikipedia.org/wiki/List_of_comedy_films_of_the_1920s

https://en.wikipedia.org/wiki/List_of_drama_films_of_the_1920s

https://en.wikipedia.org/wiki/List_of_horror_films_of_the_1920s

They basically spelled out what we want.

---

# Adventure Films

https://en.wikipedia.org/wiki/Lists_of_adventure_films

## XPaths

**Note** table[ ] and tr[ ] are all indexed

Title

//*[@id="mw-content-text"]/div[1]/table[2]/tbody/tr[2]/td[1]

//*[@id="mw-content-text"]/div[1]/table[2]/tbody/tr[8]/td[1]

**td[1]**

Country

//*[@id="mw-content-text"]/div[1]/table[2]/tbody/tr[5]/td[4]

//*[@id="mw-content-text"]/div[1]/table[3]/tbody/tr[8]/td[4]

//*[@id="mw-content-text"]/div[1]/table[11]/tbody/tr[13]/td[4]

**td[4]**

Subgenre

//*[@id="mw-content-text"]/div[1]/table[11]/tbody/tr[2]/td[5]

//*[@id="mw-content-text"]/div[1]/table[10]/tbody/tr[16]/td[5]

**td[5]**

I will need to enumerate
1. The Decades
2. The Table Numbers
3. The Row Indices

In [16]:
# Decades

decades = np.arange(1950,2021,10)
decades

array([1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020])

Table Numbers

First table //*[@id="mw-content-text"]/div[1]/table[2]/tbody/tr[2]/td[1]/i/b/a

Last Table //*[@id="mw-content-text"]/div[1]/table[11]/tbody/tr[26]/td[1]

In [9]:
# Table Numbers

tables_ = np.arange(1,12,1)
tables_

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [10]:
# Row Indices
# Ball Park 50

rows_ = np.arange(1,51,1)
rows_

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])

Base xpath 

//*[@id="mw-content-text"]/div[1]/table[??]/tbody/tr[??]/

Base Website

https://en.wikipedia.org/wiki/List_of_adventure_films_of_the_1990s

In [19]:
adventure_movies = []

In [21]:
wd = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\redoc\AppData\Local\Temp\ipykernel_157988\3832095755.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(ChromeDriverManager().install())


In [22]:
for decade_ in decades:
    wd.get('https://en.wikipedia.org/wiki/List_of_adventure_films_of_the_'+str(decade_)+'s')

    for table_ in tables_:
        for row_ in rows_:
            adventure_movie = {}
            try:
                adventure_movie['Title'] = wd.find_element("xpath", "//*[@id='mw-content-text']/div[1]/table["+str(table_)+"]/tbody/tr["+str(row_)+"]/"+"td[1]").text
                adventure_movie['Country'] = wd.find_element("xpath", "//*[@id='mw-content-text']/div[1]/table["+str(table_)+"]/tbody/tr["+str(row_)+"]/"+"td[4]").text
                adventure_movie['Subgenre'] = wd.find_element("xpath", "//*[@id='mw-content-text']/div[1]/table["+str(table_)+"]/tbody/tr["+str(row_)+"]/"+"td[5]").text
            except:
                pass
            adventure_movies.append(adventure_movie)

In [23]:
adventure_movies_df = pd.DataFrame(adventure_movies)

In [24]:
adventure_movies_df.dropna(axis = 0, how = 'all', inplace = True) # drop rows with all NaN values

In [27]:
adventure_movies_df['Genre']='Adventure'

In [28]:
adventure_movies_df.head(10)

,Title,Country,Subgenre,Genre
1,,NaN,NaN,Adventure
3,Before 1920\n1920s\n1930s\n1940s\n1950s\n1960s...,NaN,NaN,Adventure
4,vte,NaN,NaN,Adventure
51,Abbott and Costello in the Foreign Legion,United States,Adventure comedy[1],Adventure
52,The Avengers,United States,[2][3],Adventure
53,Les Aventuriers de l'air [fr],France,[4][unreliable source?],Adventure
54,The Bandit Queen,United States,Western[5],Adventure
55,Bitter Springs,Australia\nUnited Kingdom,[6],Adventure
56,Black Jack,France\nUnited States\nSpain,Sea adventure[7][unreliable source?],Adventure
57,The Black Rose,United States,[8],Adventure


In [29]:
adventure_movies_df.to_csv('Adventure_Movie_Scrape_Final.csv')

---

# Comedy Films

https://en.wikipedia.org/wiki/Lists_of_comedy_films

## XPaths

**Note** table[ ] and tr[ ] are all indexed

Title

//*[@id="mw-content-text"]/div[1]/table[3]/tbody/tr[8]/td[1]

//*[@id="mw-content-text"]/div[1]/table[3]/tbody/tr[17]/td[1]

**td[1]**

Country

//*[@id="mw-content-text"]/div[1]/table[3]/tbody/tr[14]/td[4]

//*[@id="mw-content-text"]/div[1]/table[3]/tbody/tr[35]/td[4]

**td[4]**

Subgenre

//*[@id="mw-content-text"]/div[1]/table[3]/tbody/tr[15]/td[5]

//*[@id="mw-content-text"]/div[1]/table[3]/tbody/tr[69]/td[5]

**td[5]**

In [30]:
comedy_movies = []

In [31]:
wd = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\redoc\AppData\Local\Temp\ipykernel_157988\3832095755.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(ChromeDriverManager().install())


In [32]:
for decade_ in decades:
    wd.get('https://en.wikipedia.org/wiki/List_of_comedy_films_of_the_'+str(decade_)+'s')

    for table_ in tables_:
        for row_ in rows_:
            comedy_movie = {}
            try:
                comedy_movie['Title'] = wd.find_element("xpath", "//*[@id='mw-content-text']/div[1]/table["+str(table_)+"]/tbody/tr["+str(row_)+"]/"+"td[1]").text
                comedy_movie['Country'] = wd.find_element("xpath", "//*[@id='mw-content-text']/div[1]/table["+str(table_)+"]/tbody/tr["+str(row_)+"]/"+"td[4]").text
                comedy_movie['Subgenre'] = wd.find_element("xpath", "//*[@id='mw-content-text']/div[1]/table["+str(table_)+"]/tbody/tr["+str(row_)+"]/"+"td[5]").text
            except:
                pass
            comedy_movies.append(comedy_movie)

In [33]:
comedy_movies_df = pd.DataFrame(comedy_movies)

comedy_movies_df.dropna(axis = 0, how = 'all', inplace = True) # drop rows with all NaN values

comedy_movies_df['Genre']='Comedy'

comedy_movies_df.head()

comedy_movies_df.to_csv('Comedy_Movie_Scrape_Final.csv')

---

# Drama Films

https://en.wikipedia.org/wiki/List_of_drama_films

The drama films listed are not intable format. Scrape last.

---

# Horror Films

https://en.wikipedia.org/wiki/Lists_of_horror_films

URL Formats

https://en.wikipedia.org/wiki/List_of_horror_films_of_2000

https://en.wikipedia.org/wiki/List_of_horror_films_of_1984

They end by the year

In [6]:
# Enumerate Years

years_ = np.arange(1960,2022,1)
years_

array([1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970,
       1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981,
       1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992,
       1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003,
       2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014,
       2015, 2016, 2017, 2018, 2019, 2020, 2021])

XPath

Title

//*[@id="mw-content-text"]/div[1]/table[2]/tbody/tr[1]/th

//*[@id="mw-content-text"]/div[1]/table[2]/tbody/tr[7]/th 

//*[@id="mw-content-text"]/div[1]/table[2]/tbody/tr[11]/th

Country

//*[@id="mw-content-text"]/div[1]/table[2]/tbody/tr[11]/td[3]

//*[@id="mw-content-text"]/div[1]/table[2]/tbody/tr[33]/td[3]

Note: No subgenre

In [3]:
horror_movies = []

In [4]:
wd = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\redoc\AppData\Local\Temp\ipykernel_179044\3832095755.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(ChromeDriverManager().install())


In [11]:
for year_ in years_:
    wd.get('https://en.wikipedia.org/wiki/List_of_horror_films_of_'+str(year_))

    for table_ in tables_:
        for row_ in rows_:
            horror_movie = {}
            try:
                horror_movie['Title'] = wd.find_element("xpath", "//*[@id='mw-content-text']/div[1]/table["+str(table_)+"]/tbody/tr["+str(row_)+"]/"+"th").text
                horror_movie['Country'] = wd.find_element("xpath", "//*[@id='mw-content-text']/div[1]/table["+str(table_)+"]/tbody/tr["+str(row_)+"]/"+"td[3]").text
                # horror_movie['Subgenre'] = wd.find_element("xpath", "//*[@id='mw-content-text']/div[1]/table["+str(table_)+"]/tbody/tr["+str(row_)+"]/"+"td[5]").text
            except:
                pass
            horror_movies.append(horror_movie)

In [12]:
horror_movies_df = pd.DataFrame(horror_movies)

horror_movies_df.dropna(axis = 0, how = 'all', inplace = True) # drop rows with all NaN values

horror_movies_df['Genre']='horror'

horror_movies_df['Subgenre']=np.nan

horror_movies_df.head()

horror_movies_df.to_csv('horror_Movie_Scrape_Final.csv')

---